# Practical example. Audiobooks

## Create the machine learning algorithm



### Import the relevant libraries

In [1]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

### Data

In [5]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float64)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int64)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int64)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.int64)

### Model
Outline, optimizers, loss, early stopping and training

In [7]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=4)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 1s - 34ms/step - accuracy: 0.6614 - loss: 0.6072 - val_accuracy: 0.8009 - val_loss: 0.4988
Epoch 2/100
36/36 - 0s - 2ms/step - accuracy: 0.7516 - loss: 0.4827 - val_accuracy: 0.8210 - val_loss: 0.4108
Epoch 3/100
36/36 - 0s - 2ms/step - accuracy: 0.7740 - loss: 0.4296 - val_accuracy: 0.8121 - val_loss: 0.3757
Epoch 4/100
36/36 - 0s - 2ms/step - accuracy: 0.7801 - loss: 0.4016 - val_accuracy: 0.8188 - val_loss: 0.3563
Epoch 5/100
36/36 - 0s - 2ms/step - accuracy: 0.7863 - loss: 0.3851 - val_accuracy: 0.8210 - val_loss: 0.3464
Epoch 6/100
36/36 - 0s - 2ms/step - accuracy: 0.8033 - loss: 0.3729 - val_accuracy: 0.8166 - val_loss: 0.3342
Epoch 7/100
36/36 - 0s - 2ms/step - accuracy: 0.8041 - loss: 0.3650 - val_accuracy: 0.8210 - val_loss: 0.3282
Epoch 8/100
36/36 - 0s - 2ms/step - accuracy: 0.8086 - loss: 0.3578 - val_accuracy: 0.8143 - val_loss: 0.3318
Epoch 9/100
36/36 - 0s - 2ms/step - accuracy: 0.8150 - loss: 0.3520 - val_accuracy: 0.8098 - val_loss: 0.3252
Epoch 10/

## Testing 

In [10]:
test_loss,test_accuracy = model.evaluate(test_inputs,test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8347 - loss: 0.3493 
